In [ ]:
import requests
import random
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import sqlite3
import time
import urllib

In [ ]:
# Autores ficticios 

AUTORES_FICTICIOS = [
    "Juan Pérez", "María González", "Carlos Benítez", "Ana Rodríguez", "Luis Fernández",
    "Sofía Martínez", "Miguel López", "Valeria Ramírez", "Diego Acosta", "Laura Cáceres",
    "Fernando Duarte", "Paula Ortiz", "Ricardo Silva", "Natalia Romero", "Andrés Villalba",
    "Marcos Rojas", "Lucía Torres", "Jorge Mendoza", "Cecilia Franco", "Héctor Cabrera",
    "Patricia Aguirre", "Tomás Medina", "Carla Vega", "Esteban Morales", "Verónica Paredes",
    "Alberto Núñez", "Gabriela Suárez", "Sebastián Figueroa", "Monica Castro", "Raúl Vargas",
    "Isabel Soto", "Renato Salinas", "Lorena Ayala", "Pablo Riquelme", "Camila Espínola",
    "Santiago Cabrera", "Juliana Duarte", "Javier Peralta", "Mariana Ríos", "Diego Quiñónez",
    "Adriana Villanueva", "Felipe Chamorro", "Natalia Acuña", "Hugo Benítez", "Claudia Morel",
    "Ricardo Espinoza", "Viviana Ortiz", "Martín Cardozo", "Daniela Giménez", "Gonzalo Rojas",
    "Antonio Ledesma", "Elena Caballero", "Víctor Herrera", "Mónica Peralta", "Ignacio Farias",
    "Silvia Campos", "Alfonso Paredes", "Carolina Gutiérrez", "Joaquín Aguirre", "Luz Martínez",
    "Emiliano Torres", "Patricia Salazar", "Rodrigo Molina", "María Eugenia Díaz", "Fabián Rojas",
    "Camila Castillo", "Guillermo Bravo", "Fernanda Ramírez", "Andrés Molina", "Natalia Vega",
    "Tomás Ruiz", "Lorena Mendoza", "Diego Fernández", "Isabel López", "Juan Carlos Paredes",
    "Valentina Ríos", "Santiago Morales", "Claudia Pérez", "Ricardo Herrera", "Lucía Benítez",
    "Héctor Riquelme", "Ana Sofía Romero", "Martín González", "Daniela Cabrera", "Javier Duarte",
    "Paula Ramírez", "Fernando Castro", "Carla Ortiz", "Sebastián Silva", "Verónica López",
    "Andrés Aguirre", "Mariana Núñez", "Diego Fernández", "Natalia Torres", "Luis Rojas",
    "Sofía Cabrera", "Miguel Herrera", "Laura Benítez", "Ricardo Martínez", "Valeria Peralta",
    "Jorge Ramírez", "Cecilia Vargas", "Hugo Riquelme", "Isabel Duarte", "Tomás Gutiérrez"
]

def asignar_autor_aleatorio():
    """Devuelve un autor ficticio al azar."""
    return random.choice(AUTORES_FICTICIOS)


In [ ]:
UEL_BASE = "https://books.toscrape.com/"
DB_NAME = "libros_escrapeados.db"

#Conectamos a la base de datos
conn = sqlite3.connect(DB_NAME)
conn.execute("PRAGMA foreign_keys = ON;")
cursor = conn.cursor()